In [1]:
!pip3 install -U sentence-transformers
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.9 MB/s 
     |████████████████████████████████| 4.4 MB 31.2 MB/s 
     |████████████████████████████████| 1.2 MB 48.8 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
     |████████████████████████████████| 596 kB 50.5 MB/s 
     |████████████████████████████████| 6.6 MB 34.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6352b425130abfbe64affba2a4cb9805f8e4504ca4aecbb06014f6c2fe2a3127
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models
from sentence_transformers.losses import DenoisingAutoEncoderLoss

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
model = SentenceTransformer('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased')

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

def export_most_similar(path_input, path_output):
  f=open(path_input)
  lines=f.readlines()
  lines=[l.strip() for l in lines]
  f.close()

  result=dict()
  result_errors=list()

  for i, l in enumerate(lines):

    # print(l)
    line=eval(l)
    if i%100==0:
      print("PROCESSED {} OUT OF {}".format(i, len(lines)))

    key=line[0]

    method_tokens=" ".join(eval(line[1]))
    dict_open=eval(line[2])

    issues=list()
    issues_id=list()
    issues_ok=list() # we kept track if the issue is empty (i.e., after the processing for extracting the list of words, the list is empty)
    is_ok=False
    for k in dict_open.keys():
      issues_id.append(int(k))
      issues.append(" ".join(dict_open[k]))
      if len(" ".join(dict_open[k]).strip())>0:
        is_ok=True
        issues_ok.append(True)
      else:
        issues_ok.append(False)

    # if method is empty we choose the last issue
    if len(issues)==0:
      result[key]=-1
      continue
    if len(eval(line[1]))==0:
      result[key]=issues_id[-1]
      print("METHOD EMPTY")
      result_errors.append("METHOD EMPTY")
      continue
    if is_ok==False:
      result[key]=issues_id[-1]
      print("ISSUE EMPTY")
      result_errors.append("ISSUE EMPTY")
      continue

    issue_embeddings = model.encode(issues)

    method_embedding=model.encode([method_tokens])
    simil=cosine_similarity(
      [method_embedding[0]],
      issue_embeddings
    )
    simil=simil[0]
    # print("SIMIL")
    # print(simil)
    order=sorted(range(len(simil)), key=lambda k: simil[k], reverse=True)
    # print("ORDER")
    # print(order)
    # print("ISSUES")
    # print(issues)
    # print(issues_id)

    best_issue_id=order[0]
    # print(issues_id)
    # print(issues)
    # print(best_issue_id)
    best_issue=(issues_id[best_issue_id])
    result[key]=best_issue
  f=open(file_output, "w+")
  for k in result.keys():
    f.write("{} {}\n".format(k, result[k]))
  f.close()

  f=open(file_output.replace(".txt", "_errors.txt"), "w+")
  for k in result_errors:
    f.write("{}\n".format(k))
  f.close()

In [ ]:
for s in ["train", "eval", "test"]:
  file_input= "/content/drive/MyDrive/Sentence-Transformer-Base/tsdae_issue_{}.txt".format(s)
  file_output= "/content/drive/MyDrive/Sentence-Transformer-Base/tsdae_issue_{}_result.txt".format(s)
  export_most_similar(file_input, file_output)


PROCESSED 0 OUT OF 68215
METHOD EMPTY
METHOD EMPTY
PROCESSED 100 OUT OF 68215
PROCESSED 200 OUT OF 68215
METHOD EMPTY
